In [1]:
import pandas as pd
import numpy as np

import json
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [2]:
train_data = pd.read_csv('train_modified.csv', dtype={'fullVisitorId': 'str', 'visitId': 'str'}) # Important!!
print(train_data.shape)
train_data.head()

/Users/nomi/Documents/virtualenvs/data_analysais_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [4]:
relevantAttributes = ['totals.hits', 'visitNumber', 'geoNetwork.country', 'totals.transactionRevenue']

train_data['totals.transactionRevenue'] = train_data['totals.transactionRevenue'].fillna(0)

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)

trainData_DecisionTree = trainData[relevantAttributes]
testData_DecisionTree = testData[relevantAttributes]

trainData_DecisionTree = pd.DataFrame(trainData_DecisionTree)
testData_DecisionTree = pd.DataFrame(testData_DecisionTree)

In [5]:
#Figure out labeling for country param
unique_countries = train_data['geoNetwork.country'].unique()
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
print(countryCodeDict)

{'Anguilla': 0, 'Curaçao': 1, 'Kenya': 2, 'Venezuela': 3, 'St. Lucia': 4, 'United States': 5, 'Nigeria': 6, 'Puerto Rico': 7, 'Canada': 8, 'Guadeloupe': 9, 'Ecuador': 10, '(not set)': 11, 'Nicaragua': 12, 'Kuwait': 13, 'Japan': 14, 'Hong Kong': 15, 'Panama': 16, 'Belgium': 17, 'Indonesia': 18, 'El Salvador': 19, 'Cyprus': 20, 'Taiwan': 21, 'South Korea': 22, 'Australia': 23, 'Switzerland': 24, 'Chile': 25, 'Colombia': 26, 'Mexico': 27, 'United Arab Emirates': 28, 'Singapore': 29, 'Israel': 30, 'Armenia': 31, 'Guatemala': 32, 'Lebanon': 33, 'Portugal': 34, 'Saudi Arabia': 35, 'Ukraine': 36, 'Malaysia': 37, 'Finland': 38, 'China': 39, 'Argentina': 40, 'Greece': 41, 'United Kingdom': 42, 'Georgia': 43, 'Kazakhstan': 44, 'Spain': 45, 'Brazil': 46, 'South Africa': 47, 'France': 48, 'Germany': 49, 'Poland': 50, 'Peru': 51, 'Thailand': 52, 'Netherlands': 53, 'Ireland': 54, 'New Zealand': 55, 'Egypt': 56, 'Italy': 57, 'Philippines': 58, 'Pakistan': 59, 'Sweden': 60, 'India': 61, 'Hungary': 62,

In [6]:
y = trainData_DecisionTree["totals.transactionRevenue"]
X = trainData_DecisionTree[['totals.hits', 'visitNumber', 'geoNetwork.country']]

X['geoNetwork.country'] = X['geoNetwork.country'].transform(lambda x: countryCodeDict[x])

# X = X[['totals.hits', 'visitNumber']]
regressor = DecisionTreeRegressor(max_depth=15)
regressor.fit(X,y)

/Users/nomi/Documents/virtualenvs/data_analysais_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [7]:
y = testData_DecisionTree["totals.transactionRevenue"]

X = testData_DecisionTree[['totals.hits', 'visitNumber', 'geoNetwork.country']]

X['geoNetwork.country'] = X['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
# X = X[['totals.hits', 'visitNumber']]

y_pred = regressor.predict(X)
# regressor.score(X, y, sample_weight=None)

/Users/nomi/Documents/virtualenvs/data_analysais_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
testData['predicted'] = y_pred

y = testData.groupby('fullVisitorId')['totals.transactionRevenue'].sum()
y_pred = testData.groupby('fullVisitorId')['predicted'].sum()

y_pred = y_pred + 1
y = y + 2

y_pred_ln = np.log(y_pred)
y_true_ln = np.log(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

4.87164849624008

In [9]:
test_data = pd.read_csv('test_flat.csv', dtype={'fullVisitorId': 'str', 'visitId': 'str'}) # Important!!

print (test_data.columns.values)

# test_data['fullVisitorId'] = test_data['fullVisitorId'].astype(str)

X = test_data[['hits', 'visitNumber', 'country']]
X['country'] = X['country'].transform(lambda x: countryCodeDict.get(x, 220))
y_pred = regressor.predict(X)

y_pred += 1

test_data['predicted'] = y_pred
# test_data[['fullVisitorId', 'predicted']].to_csv('output_row.csv')

y_pred_sum = test_data.groupby('fullVisitorId')['predicted'].sum()

y_pred_log = np.log(y_pred_sum)

# y_pred_sum.to_csv('output_sum.csv')
y_pred_log.to_csv('output_log.csv')

print(y_pred_log)

['channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime' 'browser'
 'browserVersion' 'browserSize' 'operatingSystem' 'operatingSystemVersion'
 'isMobile' 'mobileDeviceBranding' 'mobileDeviceModel'
 'mobileInputSelector' 'mobileDeviceInfo' 'mobileDeviceMarketingName'
 'flashVersion' 'language' 'screenColors' 'screenResolution'
 'deviceCategory' 'continent' 'subContinent' 'country' 'region' 'metro'
 'city' 'cityId' 'networkDomain' 'latitude' 'longitude' 'networkLocation'
 'visits' 'hits' 'pageviews' 'newVisits' 'bounces' 'campaign' 'source'
 'medium' 'keyword' 'isTrueDirect' 'referralPath' 'adContent'
 'adwordsClickInfo.criteriaParameters' 'adwordsClickInfo.page'
 'adwordsClickInfo.slot' 'adwordsClickInfo.gclId'
 'adwordsClickInfo.adNetworkType' 'adwordsClickInfo.isVideoAd']


/Users/nomi/Documents/virtualenvs/data_analysais_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


fullVisitorId
0000000259678714014    15.488392
0000049363351866189     1.098612
0000053049821714864     0.000000
0000059488412965267     1.098612
0000085840370633780     0.000000
0000091131414287111     0.000000
0000117255350596610    16.895624
0000118334805178127     0.000000
0000130646294093000     0.000000
0000150005271820273     0.000000
0000166374699289385     7.894911
0000174453501096099     0.000000
000018122977590134     17.143001
000020731284570628      0.000000
0000232022622082281     0.000000
0000271086753662651     0.000000
0000282648818935742     0.000000
00003038793126460       6.192324
0000313524203455157     0.000000
0000324924635296742     0.000000
0000338548677636278     0.000000
0000354865008116989     0.000000
0000359974620542953     0.693147
0000384434116640351    17.242623
0000385653946068037     0.000000
0000397214032106948     0.000000
0000414003317636552    16.872595
0000421850492821864     0.000000
0000436683523507380    12.261993
0000443488189725694     0.000